<a href="https://colab.research.google.com/github/ShagunSharma98/Global-Structural-Earthquake-Damage-Prediction/blob/master/SHM_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reading Zipped Data from drive**

In [1]:
import numpy as n
import pandas as p
import glob
import matplotlib.pyplot

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# !unzip gdrive/My\ Drive/data/train.zip
# !unzip -q "/content/gdrive/My Drive/dataset.zip"

# root_path = 'gdrive/MyDrive/Structural Damage Prediction/'
# !ls gdrive/MyDrive/Structural Damage Prediction
!ls gdrive/MyDrive

'All set'
'Colab Notebooks'
'Copy of SHM_Notebook.ipynb'
 Copy_of_X_Y.zip
'Employee Stress Prediction Dataset'
 IMG_20200330_132432.jpg
 Mask_RCNN-master
'Mussoorie '
'Pandey, Shagun_DAA_ND.pdf'
 Please_DocuSign_Welcome_to_SHL_Shagun_Pandey.pdf
 Screenshot_2020-03-30-13-25-07-545_com.google.android.youtube.jpg
 SHM_Notebook.ipynb
'Structural Damage Prediction'
"The Data Scientist's Toolbox - Moad HANI"
'Untitled document.gdoc'


In [4]:
!unzip gdrive/MyDrive/Copy_of_X_Y.zip 

Archive:  gdrive/MyDrive/Copy_of_X_Y.zip
   creating: X & y/Test/
   creating: X & y/Test/X_test/
  inflating: X & y/Test/X_test/X_test_0.csv  
  inflating: X & y/Test/X_test/X_test_1.csv  
  inflating: X & y/Test/X_test/X_test_10.csv  
  inflating: X & y/Test/X_test/X_test_100.csv  
  inflating: X & y/Test/X_test/X_test_101.csv  
  inflating: X & y/Test/X_test/X_test_102.csv  
  inflating: X & y/Test/X_test/X_test_103.csv  
  inflating: X & y/Test/X_test/X_test_104.csv  
  inflating: X & y/Test/X_test/X_test_105.csv  
  inflating: X & y/Test/X_test/X_test_106.csv  
  inflating: X & y/Test/X_test/X_test_107.csv  
  inflating: X & y/Test/X_test/X_test_108.csv  
  inflating: X & y/Test/X_test/X_test_109.csv  
  inflating: X & y/Test/X_test/X_test_11.csv  
  inflating: X & y/Test/X_test/X_test_110.csv  
  inflating: X & y/Test/X_test/X_test_111.csv  
  inflating: X & y/Test/X_test/X_test_112.csv  
  inflating: X & y/Test/X_test/X_test_113.csv  
  inflating: X & y/Test/X_test/X_test_114.cs

In [5]:
path_1="/content/X & y/Train/"
path_2="/content/X & y/Test/"

In [6]:
cols_1=n.concatenate([n.arange(178),n.arange(179,285),n.arange(286,332),n.arange(333,440)],axis=0)

In [7]:
X_train={}
X_test={}
y_train={}
y_test={}

file_list_1=glob.glob(path_1+"X_train/"+"X_train_*.csv")
for file_path in file_list_1:
    X_train[file_path[file_path.rfind('_')+1:-4]]=p.DataFrame(p.read_csv(file_path))
    
file_list_2=glob.glob(path_1+"y_train/"+"y_train_*.csv")
for file_path in file_list_2:
    y_train[file_path[file_path.rfind('_')+1:-4]]=p.DataFrame(p.read_csv(file_path))
    
file_list_3=glob.glob(path_2+"X_test/"+"X_test_*.csv")
for file_path in file_list_3:
    X_test[file_path[file_path.rfind('_')+1:-4]]=p.DataFrame(p.read_csv(file_path))
    
file_list_4=glob.glob(path_2+"y_test/"+"y_test_*.csv")
for file_path in file_list_4:
    y_test[file_path[file_path.rfind('_')+1:-4]]=p.DataFrame(p.read_csv(file_path))

In [8]:
X_Input={}
y_Input={}
X_Output={}
y_Output={}

for key in cols_1:
    X_Input[key]=n.reshape(n.asarray(X_train[str(key)]),(X_train[str(key)].shape[0],4001,2))
    y_Input[key]=n.reshape(n.asarray(y_train[str(key)]),(y_train[str(key)].shape[0],1))
    X_Output[key]=n.reshape(n.asarray(X_test[str(key)]),(X_test[str(key)].shape[0],4001,2))
    y_Output[key]=n.reshape(n.asarray(y_test[str(key)]),(y_test[str(key)].shape[0],1))

In [9]:
from keras.models import Sequential
from keras.layers import Conv1D,MaxPool1D,Dense,Flatten
from keras.models import model_from_json

def make_cnn_model(kernel_size, filters):
  
  model=Sequential()

  model.add(Conv1D(filters=filters, kernel_size=kernel_size, input_shape=(4001,2), activation='tanh', padding="same"))
  model.add(MaxPool1D())
  model.add(Conv1D(filters=150,activation='relu',kernel_size=10))
  model.add(MaxPool1D())
  model.add(Conv1D(filters=100,activation='relu',kernel_size=10))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [10]:
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [11]:
epochs = [20, 30, 40]
kernel_size = [25, 50, 100]
filters = [300, 400, 500]
scorers = {
        'precision_score': make_scorer(precision_score),
        'recall_score': make_scorer(recall_score),
        'accuracy_score': make_scorer(accuracy_score)
        }

param_grid = dict(epochs=epochs, kernel_size=kernel_size
                  , filters=filters
                  )

In [12]:
# grid_search_cv=GridSearchCV(model,param_grid,verbose=1,cv=3,scoring=scorers,refit="precision_score")
cnn_model = KerasClassifier(build_fn=make_cnn_model, verbose=2)
grid = GridSearchCV(estimator=cnn_model, param_grid=param_grid, n_jobs=-1, verbose=2, scoring=scorers, refit="precision_score")

In [13]:
cnn_grid_result = {}

for key in cols_1:
    cnn_grid_result[key] =  grid.fit(X_Input[key], y_Input[key])


Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 11.1min finished


Epoch 1/20
2/2 - 4s - loss: 0.6931 - accuracy: 0.5333
Epoch 2/20
2/2 - 0s - loss: 0.6931 - accuracy: 0.5333
Epoch 3/20
2/2 - 0s - loss: 0.6931 - accuracy: 0.5333
Epoch 4/20
2/2 - 0s - loss: 0.6930 - accuracy: 0.5333
Epoch 5/20
2/2 - 0s - loss: 0.6930 - accuracy: 0.5333
Epoch 6/20
2/2 - 0s - loss: 0.6930 - accuracy: 0.5333
Epoch 7/20
2/2 - 0s - loss: 0.6929 - accuracy: 0.5333
Epoch 8/20
2/2 - 0s - loss: 0.6929 - accuracy: 0.5333
Epoch 9/20
2/2 - 0s - loss: 0.6929 - accuracy: 0.5333
Epoch 10/20
2/2 - 0s - loss: 0.6928 - accuracy: 0.5333
Epoch 11/20
2/2 - 0s - loss: 0.6928 - accuracy: 0.5333
Epoch 12/20
2/2 - 0s - loss: 0.6928 - accuracy: 0.5333
Epoch 13/20
2/2 - 0s - loss: 0.6928 - accuracy: 0.5333
Epoch 14/20
2/2 - 0s - loss: 0.6928 - accuracy: 0.5333
Epoch 15/20
2/2 - 0s - loss: 0.6928 - accuracy: 0.5333
Epoch 16/20
2/2 - 0s - loss: 0.6927 - accuracy: 0.5333
Epoch 17/20
2/2 - 0s - loss: 0.6927 - accuracy: 0.5333
Epoch 18/20
2/2 - 0s - loss: 0.6927 - accuracy: 0.5333
Epoch 19/20
2/2 - 0

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 11.4min finished


Epoch 1/20
2/2 - 2s - loss: 0.6408 - accuracy: 0.6333
Epoch 2/20
2/2 - 0s - loss: 0.4980 - accuracy: 0.5833
Epoch 3/20
2/2 - 0s - loss: 0.4134 - accuracy: 0.8000
Epoch 4/20
2/2 - 0s - loss: 0.2612 - accuracy: 0.9000
Epoch 5/20
2/2 - 0s - loss: 0.2493 - accuracy: 0.9167
Epoch 6/20
2/2 - 0s - loss: 0.2771 - accuracy: 0.9000
Epoch 7/20
2/2 - 0s - loss: 0.3046 - accuracy: 0.8833
Epoch 8/20
2/2 - 0s - loss: 0.1763 - accuracy: 0.9333
Epoch 9/20
2/2 - 0s - loss: 0.2069 - accuracy: 0.9167
Epoch 10/20
2/2 - 0s - loss: 0.1266 - accuracy: 0.9500
Epoch 11/20
2/2 - 0s - loss: 0.1864 - accuracy: 0.9333
Epoch 12/20
2/2 - 0s - loss: 0.1253 - accuracy: 0.9833
Epoch 13/20
2/2 - 0s - loss: 0.1353 - accuracy: 0.9667
Epoch 14/20
2/2 - 0s - loss: 0.1126 - accuracy: 0.9833
Epoch 15/20
2/2 - 0s - loss: 0.1402 - accuracy: 0.9167
Epoch 16/20
2/2 - 0s - loss: 0.1088 - accuracy: 0.9833
Epoch 17/20
2/2 - 0s - loss: 0.1245 - accuracy: 0.9667
Epoch 18/20
2/2 - 0s - loss: 0.1067 - accuracy: 0.9833
Epoch 19/20
2/2 - 0

...

Epoch 1/20
2/2 - 1s - loss: 0.8988 - accuracy: 0.5167
Epoch 2/20
2/2 - 0s - loss: 0.5113 - accuracy: 0.5833
Epoch 3/20
2/2 - 0s - loss: 0.5451 - accuracy: 0.6833
Epoch 4/20
2/2 - 0s - loss: 0.4792 - accuracy: 0.6167
Epoch 5/20
2/2 - 0s - loss: 0.4204 - accuracy: 0.7667
Epoch 6/20
2/2 - 0s - loss: 0.3290 - accuracy: 0.8833
Epoch 7/20
2/2 - 0s - loss: 0.2211 - accuracy: 0.9167
Epoch 8/20
2/2 - 0s - loss: 0.1680 - accuracy: 0.9333
Epoch 9/20
2/2 - 0s - loss: 0.0701 - accuracy: 0.9833
Epoch 10/20
2/2 - 0s - loss: 0.1244 - accuracy: 0.9333
Epoch 11/20
2/2 - 0s - loss: 0.0429 - accuracy: 0.9833
Epoch 12/20
2/2 - 0s - loss: 0.0439 - accuracy: 0.9667
Epoch 13/20
2/2 - 0s - loss: 0.0184 - accuracy: 1.0000
Epoch 14/20
2/2 - 0s - loss: 0.0142 - accuracy: 1.0000
Epoch 15/20
2/2 - 0s - loss: 0.0218 - accuracy: 1.0000
Epoch 16/20
2/2 - 0s - loss: 0.0406 - accuracy: 0.9833
Epoch 17/20
2/2 - 0s - loss: 0.0766 - accuracy: 0.9667
Epoch 18/20
2/2 - 0s - loss: 0.0947 - accuracy: 0.9667
Epoch 19/20
2/2 - 0

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 11.5min finished


Epoch 1/30
2/2 - 1s - loss: 0.8788 - accuracy: 0.4667
Epoch 2/30
2/2 - 0s - loss: 0.5650 - accuracy: 0.5333
Epoch 3/30
2/2 - 0s - loss: 0.4941 - accuracy: 0.5667
Epoch 4/30
2/2 - 0s - loss: 0.5033 - accuracy: 0.6333
Epoch 5/30
2/2 - 0s - loss: 0.4215 - accuracy: 0.7833
Epoch 6/30
2/2 - 0s - loss: 0.4194 - accuracy: 0.8000
Epoch 7/30
2/2 - 0s - loss: 0.2729 - accuracy: 0.9167
Epoch 8/30
2/2 - 0s - loss: 0.2610 - accuracy: 0.9000
Epoch 9/30
2/2 - 0s - loss: 0.1979 - accuracy: 0.9000
Epoch 10/30
2/2 - 0s - loss: 0.1371 - accuracy: 0.9000
Epoch 11/30
2/2 - 0s - loss: 0.1479 - accuracy: 0.9167
Epoch 12/30
2/2 - 0s - loss: 0.0978 - accuracy: 0.9500
Epoch 13/30
2/2 - 0s - loss: 0.0849 - accuracy: 0.9333
Epoch 14/30
2/2 - 0s - loss: 0.0546 - accuracy: 0.9833
Epoch 15/30
2/2 - 0s - loss: 0.0330 - accuracy: 1.0000
Epoch 16/30
2/2 - 0s - loss: 0.0966 - accuracy: 0.9667
Epoch 17/30
2/2 - 0s - loss: 0.0710 - accuracy: 0.9667
Epoch 18/30
2/2 - 0s - loss: 0.0209 - accuracy: 1.0000
Epoch 19/30
2/2 - 0

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

In [14]:
for key in cols_1:
    print("Best: %f using %s" % (cnn_grid_result[key].best_score_, cnn_grid_result[key].best_params_))
    

Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}
Best: 0.966667 using {'epochs': 30, 'filters': 400, 'kernel_size': 50}


KeyError: ignored

In [16]:
Tuned_params = {}

for key in cols_1:
    Tuned_params[key] = {'Score': cnn_grid_result[key].best_score_,
                         'Best_params': cnn_grid_result[key].best_params_,
                         'Precision_score':cnn_grid_result[key].cv_results_['mean_test_precision_score'],
                         'Accuracy_score' : cnn_grid_result[key].cv_results_['mean_test_accuracy_score'],
                         'Recall_score' :  cnn_grid_result[key].cv_results_['mean_test_recall_score']}


KeyError: ignored

In [21]:
import pickle

In [22]:
Tuned_file = open("Tuned_parameter.pkl", "wb")
pickle.dump(Tuned_params, Tuned_file)
Tuned_file.close()


In [23]:
Tuned_file = open("Tuned_parameter.pkl", "rb")
output = pickle.load(Tuned_file)
print(output)

{0: {'Score': 0.9666666666666668, 'Best_params': {'epochs': 30, 'filters': 400, 'kernel_size': 50}, 'Precision_score': array([0.82285714, 0.89809524, 0.90952381, 0.9       , 0.875     ,
       0.87666667, 0.88666667, 0.89809524, 0.9047619 , 0.93333333,
       0.86952381, 0.90952381, 0.93333333, 0.96666667, 0.91666667,
       0.93333333, 0.84666667, 0.87619048, 0.84095238, 0.87666667,
       0.93333333, 0.90952381, 0.96666667, 0.93333333, 0.86952381,
       0.90952381, 0.86      ]), 'Accuracy_score': array([0.86666667, 0.93333333, 0.9       , 0.86666667, 0.9       ,
       0.9       , 0.9       , 0.93333333, 0.9       , 0.88333333,
       0.83333333, 0.83333333, 0.86666667, 0.88333333, 0.93333333,
       0.88333333, 0.86666667, 0.9       , 0.86666667, 0.91666667,
       0.91666667, 0.91666667, 0.93333333, 0.93333333, 0.86666667,
       0.9       , 0.85      ]), 'Recall_score': array([0.96      , 0.96      , 0.9047619 , 0.87428571, 0.96      ,
       0.93142857, 0.96      , 0.96      , 0

In [24]:
Tuned_file.close()

Saving the model for future refernces

**SHM RETRAINING**

In [10]:
model_1 = make_cnn_model(50, 300)

In [11]:
Network_1=[]

for key in cols_1:
    print('Training of CNN for Joint: '+str(key+1)+' initiated')
    model_1.fit(x=X_Input[key],y=y_Input[key],batch_size=15,validation_data=(X_Output[key], y_Output[key]),epochs=30)
    Network_1.append(model_1)
    print('Training of CNN for Joint: '+str(key+1)+' completed')
    print("\n")

# Try training with 20 epochs next time expected to be followed by nearly same result

Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 48ms/step - loss: 0.2121 - accuracy: 1.0000 - val_loss: 7.1727 - val_accuracy: 0.8125
Epoch 29/30
4/4 [==============================] - 0s 55ms/step - loss: 0.2114 - accuracy: 1.0000 - val_loss: 7.1898 - val_accuracy: 0.8125
Epoch 30/30
4/4 [==============================] - 0s 48ms/step - loss: 0.2106 - accuracy: 1.0000 - val_loss: 7.2019 - val_accuracy: 0.8125
Training of CNN for Joint: 362 completed


Training of CNN for Joint: 363 initiated
Epoch 1/30
4/4 [==============================] - 0s 73ms/step - loss: 0.3631 - accuracy: 0.9833 - val_loss: 10.1804 - val_accuracy: 0.8125
Epoch 2/30
4/4 [==============================] - 0s 48ms/step - loss: 0.2091 - accuracy: 1.0000 - val_loss: 10.3316 - val_accuracy: 0.8125
Epoch 3/30
4/4 [==============================] - 0s 49ms/step - loss: 0.2083 - accuracy: 1.0000 - val_loss: 10.4343 - val_accuracy: 0.8125
Epoch 4/30
4/4 [=====================

In [12]:
scores_1=0
efficiency_1=0
for key,index in zip(cols_1,range(437)):
    scores_1=scores_1+Network_1[index].evaluate(X_Output[key],y_Output[key])[1]
efficiency_1=(scores_1/n.shape(Network_1)[0])*100
print("\nEfficiency Obtained from 1D CNN Network on Test Dataset is: "+ str(efficiency_1)+"%")

1/1 [==============================] - 0s 26ms/step - loss: 136.0745 - accuracy: 0.6875

Efficiency Obtained from 1D CNN Network on Test Dataset is: 78.7471395881007%


In [25]:
!mkdir 1D_CNN
from keras.models import load_model

In [30]:
for key, index in zip(cols_1, range(437)):
  Network_1[index].save('1D_CNN/Model_Joint_'+str(key)+'.h5')

In [ ]:
# model = load_model('my_model.h5')

In [31]:
import shutil
shutil.make_archive('1D_CNN', 'zip', '1D_CNN')

'/content/1D_CNN.zip'

In [39]:
# !gsutil -q -m cp -r gs://my-bucket-name drive /content/drive/My\ Drive/

!cp "1D_CNN.zip" -r "gdrive/MyDrive/"